In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import os

In [4]:
cwd = os.getcwd()

In [5]:
static = pd.read_pickle(cwd + '/processed_data/static_data.pkl')
pop = static[['fips', 'Population']]

In [6]:
hosp = pd.read_pickle('/Users/hongrudu/Documents/Projects/COVIDLLM/data/processed_data/hospitalization_daily_state.pkl')
hosp['Week'] = hosp['Date'].dt.to_period('W')
hosp_week = hosp.drop(columns = ['Date', 'State', 'Temporal_resolution']).groupby(['state_name', 'Variable', 'fips', 'Week', 'Description']).sum().reset_index()
hosp_week = hosp_week.merge(pop)
hosp_week['hospitalization_per_100k'] = (hosp_week['value']/hosp_week['Population'])*100000
hosp_week = hosp_week.drop(columns = {'Variable', 'Description'})
hosp_week['Week_start'] = hosp_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week['Week_end'] = hosp_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week = hosp_week.rename(columns = {'value' : 'hospitalization'})

In [7]:
api_key = "Your Key"
openai.api_key = api_key

In [8]:
hosp_week['hospitalization_per_100k'] = round(hosp_week['hospitalization_per_100k'], 1)

In [9]:
for index, row in hosp_week[hosp_week['Week_end'] >= '2021-01-01'].iterrows():
    prompt = str(hosp_week.loc[index-4, 'hospitalization_per_100k']) +\
    ', ' + str(hosp_week.loc[index-3, 'hospitalization_per_100k']) +\
    ', ' + str(hosp_week.loc[index-2, 'hospitalization_per_100k']) +\
    ', ' + str(hosp_week.loc[index-1, 'hospitalization_per_100k']) + ', and ' +\
    str(hosp_week.loc[index, 'hospitalization_per_100k'])
    hosp_week.loc[index, 'hospitalization_per_100k_past_5'] = prompt
hosp_week_selected = hosp_week[(hosp_week['Week_end'] >= '2021-01-01')\
         &(hosp_week['Week_end'] <= '2023-01-10')].reset_index().drop(columns = ['index'])

In [10]:
for index, row in hosp_week_selected.iterrows():
    prompt =  "Describe all information in this time-series: " + \
    hosp_week_selected['hospitalization_per_100k_past_5'][index]
    
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      temperature = 0.8,
      max_tokens = 200,
      seed=1,
      messages = [
        {"role": "system", 
         "content": "You are a helpful assistant for summarizing the key Trends, Averages, Volatility, and Rate of Change in the most recent five weeks of COVID-19 hospitalization per 100K in a concise written format (one Paragraph)."},
        {"role": "user", "content" : prompt}
      ]
    )
    
    hosp_week_selected.loc[index, 'hospitalization_per_100k_gpt_trend'] = completion.choices[0].message.content
    print(completion.choices[0].message.content)
    

Over the most recent five weeks, COVID-19 hospitalization rates per 100K have shown an increasing trend, with figures of 40.9, 47.7, 54.9, 58.1, and 61.2. The average hospitalization rate over this period is approximately 52.6 per 100K. There is an increasing volatility in the rates as the numbers fluctuate more with each passing week. The rate of change also indicates a consistent upward trend, with the hospitalization rate per 100K rising by an average of 4.1 units per week over the five-week period.
Over the most recent five weeks, the COVID-19 hospitalization rate per 100K population has shown a clear increasing trend. The average hospitalization rate during this period is 56.7. The data points have exhibited moderate volatility, with a range of 13.9 from the lowest to the highest value. The rate of change in hospitalization has been steady, with an average increase of 2.56 per week.
Over the most recent five weeks, the COVID-19 hospitalization rate per 100K has shown a general upw

In [ ]:
hosp_week_selected.to_pickle(cwd + '/processed_data/hosp_gpt_trend.pkl')

### Case

In [14]:
cases = pd.read_pickle(cwd + '/processed_data/confimed_cases_daily_state.pkl')

In [15]:
cases['Week'] = cases['Date'].dt.to_period('W')
cases_week = cases.drop(columns = ['Date'])\
.groupby(['Week', 'state_name', 'fips','Description']).sum().reset_index()
cases_week['Week_start'] = cases_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
cases_week['Week_end'] = cases_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')

cases_week = cases_week.rename(columns = {'value' : 'Reported_cases'}).drop(columns = ['Description'])



/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_31269/3838927709.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cases_week = cases.drop(columns = ['Date'])\


In [17]:
df = hosp_week.merge(cases_week)

In [19]:
df = df.rename(columns = {'Population' : 'population', 'Reported_cases' : 'reported_cases'})
df['reported_cases_per_100k'] = (df['reported_cases']/df['population'])*100000
df['reported_cases_per_100k'] = round(df['reported_cases_per_100k'],1)
cases_selected = df[['state_name', 'fips', 'Week', 'Week_start', 'Week_end', 'reported_cases_per_100k']]
cases_selected = cases_selected[(cases_selected['Week_end'] >= '2021-01-01')\
         &(cases_selected['Week_end'] <= '2023-01-10')].reset_index().drop(columns = ['index'])

In [ ]:
for index, row in cases_selected.iterrows():
    prompt =  "Describe all information in this time-series: " + \
    cases_selected['reported_cases_per_100k_past_5'][index]
    
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      temperature = 1,
      max_tokens = 300,
      seed=1,
      messages = [
        {"role": "system", 
         "content": "You are a helpful assistant for summarizing the key Trends, Averages, Anomalies, and Rate of Change in the most recent five weeks of COVID-19 reported cases per 100K in a concise written format (one Paragraph)."},
        {"role": "user", "content" : prompt}
      ]
    )
    
    cases_selected.loc[index, 'reported_cases_per_100k_gpt_trend'] = completion.choices[0].message.content
    
    
    

In [ ]:
cases_selected.to_pickle(cwd + '/processed_data/cases_gpt_trend.pkl')